# Needed imports
Importing all the packages we need for the data we'll be using

In [1]:
# web scraping
from bs4 import BeautifulSoup
import requests

In [16]:
# below is the url where all the data lives
url = 'https://s3.amazonaws.com/tripdata/'

response = requests.get(url)

soup = BeautifulSoup(response.text, 'xml')

all_data_files = soup.find_all('Key')

print(f'There are {len(all_data_files)} total files.')

There are 227 total files.


In [25]:
# Create a function to download files
def download_file(url, save_path):
    r = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(r.content)

# Here we'll only grab NYC rides. We'll do this by first checking that the file is a .zip file and then ensuring that it doesn't contain 'JC' in the title.
for each_file in all_data_files:
    temp_file_name  = each_file.get_text()
    temp_filetype   = temp_file_name[-4:]
    
    if temp_filetype == '.zip' and not('JC' in temp_file_name):
        print(f'Downloading {temp_file_name}...')
        
        download_file(
            url + temp_file_name,
            'data/raw/'
        )
    
    else:
        print(f'Skpping {each_file}.')

<Key>201306-citibike-tripdata.zip</Key>


In [35]:
each.get_text()

'201306-citibike-tripdata.zip'